In [1]:
# !pip install pennylane-lightning[gpu]
# !pip uninstall pennylane -y
# !pip install pennylane --upgrade
# !pip install matplotlib


In [2]:
import pennylane as qml
import matplotlib.pyplot as plt
from pennylane.optimize import AdamOptimizer
import time
import numpy as np
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as qml_np
import pennylane as qml
import matplotlib.pyplot as plt
import math

In [3]:
# seed=42
seed = int(time.time())
np.random.seed(seed)
print("Seed:", seed)
num_qubits_factoring = 14
# num_qubits = 7
num_qubits = 14
n_layers = 2

Seed: 1691626905


In [4]:
#===================================================
import subprocess
import tracemalloc

# GPU mem usage
def get_gpu_memory_usage():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'], capture_output=True)
    output = result.stdout.decode('utf-8').strip()
    memory_usages = output.split('\n')
    total_memory_usage = sum(map(int, memory_usages))
    return total_memory_usage
# CPU mem Usage
tracemalloc.start()

In [5]:
def vqs_factoring(n_layers,n_shots=None,device_name1='default.qubit' ,device_name2='default.qubit' ): #True, remainings
# def vqs_factoring(n_layers,n_shots=None,device_name1='lightning.gpu' ,device_name2='lightning.gpu' ): #True, remainings

    print('run VQS n_layers=',n_layers,' n_shots=',n_shots,'device1 = ',device_name1,'device2 = ',device_name2)

    val_global = []

    
    # vqs
    num_qubit_vqs = 1+num_qubits
    eps_val_q = 1/math.sqrt(2**num_qubit_vqs)/100
    eps_val = min(1e-10, eps_val_q)
    tiny_change_threshold = 1e-4
    cnt_threshold_no_change = 5
    

    N = 2**(num_qubit_vqs-2)
    normal_val = math.sqrt(1/N)


    def oracle_builder_for_HT_HTZ():
        # for i in range(2,8):
        #     qml.Hadamard(wires=i)
        qml.Hadamard(wires=2)
        qml.Hadamard(wires=5)
        
        qml.CNOT(wires=[2,5])
        qml.CNOT(wires=[5,8])
        qml.CNOT(wires=[2,5])
        
        qml.Toffoli(wires=[2,6,9])
        qml.Toffoli(wires=[3,5,10])
        qml.PauliX(wires=9)
        qml.PauliX(wires=10)
        qml.Toffoli(wires=[9,10,11])
        qml.PauliX(wires=9)
        qml.PauliX(wires=10)
        qml.Toffoli(wires=[2,6,9])
        qml.Toffoli(wires=[3,5,10])
        
        qml.Toffoli(wires=[8,11,12])
        
        qml.Toffoli(wires=[2,7,9])
        qml.Toffoli(wires=[4,5,10])
        qml.PauliX(wires=9)
        qml.PauliX(wires=10)
        qml.Toffoli(wires=[9,10,13])
        qml.PauliX(wires=9)
        qml.PauliX(wires=10)
        
        qml.Toffoli(wires=[12,13,14])
        # Entangle to label qubit
        qml.CNOT([14,1])

    def oracle_builder_for_no_HT_HTZ():
        # for i in range(1,7):
        #     qml.Hadamard(wires=i)
        
        qml.Hadamard(wires=1)
        qml.Hadamard(wires=4)

        qml.CNOT(wires=[1,4])
        qml.CNOT(wires=[4,7])
        qml.CNOT(wires=[1,4])
        
        qml.Toffoli(wires=[1,5,8])
        qml.Toffoli(wires=[2,4,9])
        qml.PauliX(wires=8)
        qml.PauliX(wires=9)
        qml.Toffoli(wires=[8,9,10])
        qml.PauliX(wires=8)
        qml.PauliX(wires=9)
        qml.Toffoli(wires=[1,5,8])
        qml.Toffoli(wires=[2,4,9])
        
        qml.Toffoli(wires=[7,10,11])
        
        qml.Toffoli(wires=[1,6,8])
        qml.Toffoli(wires=[3,4,9])
        qml.PauliX(wires=8)
        qml.PauliX(wires=9)
        qml.Toffoli(wires=[8,9,12])
        qml.PauliX(wires=8)
        qml.PauliX(wires=9)
        
        qml.Toffoli(wires=[11,12,13])
        # Entangle to label qubit
        qml.CNOT([13,0])
        

    def layer_t3_no_HT(theta, qubit_posi):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # length of qubit_posi: num_qubit_vqs-1
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i], wires=(qubit_posi[i]))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            #         qml.ctrl(qml.PauliZ(qubit_posi[i+1]), qubit_posi[i]) # CZ struct2
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i+num_qubit_vqs-1], wires=(qubit_posi[i]))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            #         qml.ctrl(qml.PauliZ(qubit_posi[i+1]), qubit_posi[i]) # CZ struct2
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
    #     qml.ctrl(qml.PauliZ(qubit_posi[0]), qubit_posi[-1]) # CZ struct2
        qml.CNOT(wires=(qubit_posi[-1], qubit_posi[0]))  # CNOT struct3


    def layer_t3_with_HT(theta, num_qubit_vqs):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i], wires=(0, i+1))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            #         qml.ctrl(qml.PauliZ(i+2), (0, i+1)) # CZ struct2
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3

        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i+num_qubit_vqs-1], wires=(0, i+1))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            #         qml.ctrl(qml.PauliZ(i+2), (0, i+1)) # CZ struct2
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3
    #     qml.ctrl(qml.PauliZ(1), (0, num_qubit_vqs-1)) # CZ struct2
        qml.Toffoli(wires=(0, num_qubit_vqs-1, 1))  # CCNOT struct3





    # dev_with_HT=qml.device(device_name2, wires=num_qubit_vqs+1) #AerDevice(wires=num_qubit_vqs, shots=20000, backend='qasm_simulator')
    dev_with_HT = qml.device(device_name1, wires=num_qubits_factoring+1,shots=n_shots)


    @qml.qnode(dev_with_HT)
    def quantum_circuit_with_HT(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.Hadamard(0)
        
        return qml.expval(qml.PauliZ(0))

    # dev_with_HTZ=qml.device(device_name2, wires=num_qubit_vqs+1) #AerDevice(wires=num_qubit_vqs, shots=20000, backend='qasm_simulator')
    dev_with_HTZ = qml.device(device_name1, wires=num_qubits_factoring+1,shots=n_shots)


    @qml.qnode(dev_with_HTZ)
    def quantum_circuit_with_HTZ(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.CZ([0, 1])
        qml.Hadamard(0)
        return qml.expval(qml.PauliZ(0))


    dev_no_HT_S = qml.device(device_name2, wires=num_qubits_factoring,shots=n_shots)
    @qml.qnode(dev_no_HT_S)
    def quantum_circuit_no_HT_return_state(theta):

        oracle_builder_for_no_HT_HTZ()

        for theta_i in theta:
            layer_t3_no_HT(theta_i, list(range(num_qubit_vqs-1)))
        
        return qml.probs(wires=[i for i in range(0,7)])
        return qml.state()


    def objective_fn(theta):
        # print(qml.draw(quantum_circuit_no_HT_return_state)(theta))
        # print(qml.draw(quantum_circuit_with_HT)(theta))
        # print(qml.draw(quantum_circuit_with_HTZ)(theta))
        # raise KeyboardInterrupt
        # global val_global
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val1_1 = val1_1/normal_val
        val1_2 = val1_2/normal_val
        obj = -0.5*(val1_1 - val1_2)
        val_global.append(
            [val1_1._value.tolist(), val1_2._value.tolist(), obj._value.tolist()])
        
        return obj


    iter_max = 300  # 300
    num_of_layers = n_layers
    obj_list_rep = []
    theta_list = []
    iter_terminate_list = []

    print_flag = True

    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta = qml_np.random.uniform(
        0, 2*math.pi, size=(num_of_layers, 2*(num_qubit_vqs-1)), requires_grad=True)
    obj_list = []
    tiny_change_cnt = 0
    break_flag = False
    iter_terminate = iter_max
    for iter in range(1, iter_max+1):
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = val_global[-1][0]
        val1_2 = val_global[-1][1]
        if iter >= 2:
            val1_1_old = val_global[-2][0]
            val1_2_old = val_global[-2][1]
        else:
            val1_1_old = 999
            val1_2_old = 999
        val1 = val1_1 - val1_2
        val1_old = val1_1_old - val1_2_old
        if abs(val1) > eps_val:  # eps_val=1e-10
            if abs((val1-val1_old)/val1) < tiny_change_threshold:  # 1e-3
                tiny_change_cnt += 1
            else:
                tiny_change_cnt = 0
        if tiny_change_cnt >= cnt_threshold_no_change:  # no change for a consequtive of 5 iterations, then break
            break_flag = True
        if (iter == 1 or iter % 10 == 0 or iter == iter_max) and print_flag:
            #TODO ? 
            if (np.isclose(val1_1-val1_2,0)):
                return False
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} :: val1_2={val1_2:12.8f} :: -0.5*(val1_1 - val1_2)={-0.5*(val1_1 - val1_2):12.8f}')

        obj_list.append(obj)
        if break_flag:
            iter_terminate = iter
            break
    theta_list.append(theta)
    obj_list_rep.append(obj_list)

    val_global = []  # reset to empty

    # display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
 
    # prb = [i.item()**2 for i in state]
    iter_terminate_list.append(iter_terminate)

    # return True,remainings
    # return np.linalg.norm(prb)
    return state

In [6]:
prb = vqs_factoring(n_layers=2)
# print(prb**2)
# print(prb)

run VQS n_layers= 2  n_shots= None device1 =  default.qubit device2 =  default.qubit
iter=  1 :: obj= -1.94335179 :: val1_1=  2.65136443 :: val1_2= -1.23533915 :: -0.5*(val1_1 - val1_2)= -1.94335179
iter= 10 :: obj=-20.89542629 :: val1_1= 20.74071195 :: val1_2=-21.05014062 :: -0.5*(val1_1 - val1_2)=-20.89542629
iter= 20 :: obj=-41.00724587 :: val1_1= 41.23761934 :: val1_2=-40.77687239 :: -0.5*(val1_1 - val1_2)=-41.00724587
iter= 30 :: obj=-44.19417526 :: val1_1= 44.12100889 :: val1_2=-44.26734163 :: -0.5*(val1_1 - val1_2)=-44.19417526
iter= 40 :: obj=-44.90500573 :: val1_1= 44.90933757 :: val1_2=-44.90067389 :: -0.5*(val1_1 - val1_2)=-44.90500573
iter= 50 :: obj=-45.14107779 :: val1_1= 45.14221023 :: val1_2=-45.13994535 :: -0.5*(val1_1 - val1_2)=-45.14107779
iter= 60 :: obj=-45.19068975 :: val1_1= 45.19245494 :: val1_2=-45.18892456 :: -0.5*(val1_1 - val1_2)=-45.19068975
[6.61290476e-02 2.60782267e-03 4.40254880e-02 4.35578206e-06
 4.91674699e-02 4.33416722e-03 1.49847319e-02 9.29245212

In [7]:
# print(np.linalg.norm(prb))
for i in range(len(prb)):
    prb[i] = np.linalg.norm(prb[i])
    if prb[i]>0.1:
        print(prb[i])
        print(i)
        print(bin(i))

print(np.max(prb))
# print(prb)

0.2678968854726538
68
0b1000100
0.27164361409597887
96
0b1100000
0.27164361409597887
[6.61290476e-02 2.60782267e-03 4.40254880e-02 4.35578206e-06
 4.91674699e-02 4.33416722e-03 1.49847319e-02 9.29245212e-06
 2.18457981e-06 1.26763874e-06 4.68533306e-06 2.11249537e-06
 3.28840121e-05 2.62196335e-07 1.51789317e-05 1.99505478e-09
 2.87462372e-05 1.14206339e-06 1.19048507e-05 3.65011071e-09
 1.97941379e-05 1.05403905e-05 2.72640092e-06 5.72427387e-09
 1.26058136e-08 3.04134527e-07 1.68051384e-07 2.50127767e-07
 1.54310060e-06 1.19465370e-09 5.63399418e-07 8.22308390e-10
 4.55181115e-02 3.68870494e-03 1.42399055e-02 8.40311280e-06
 6.28419823e-02 3.75873153e-02 5.65883449e-03 1.98501205e-05
 1.82812833e-05 7.23224747e-06 4.17619334e-06 4.86179682e-09
 9.78028909e-08 7.15510150e-11 8.90869118e-08 1.98604130e-06
 2.42260743e-05 6.55935419e-07 1.47096564e-05 1.76296561e-09
 1.23352588e-05 1.73894227e-06 3.79201195e-06 2.41849515e-09
 5.77299111e-07 1.38316284e-07 2.38857481e-07 6.00307330e-10
